This python script convert .grib2 files into .tif format

In [ ]:
#import libraries
from google.colab import files
import os
import subprocess
import glob

In [ ]:
# Install GDAL if not already installed (This could take few minutes)
!apt-get install -y gdal-bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 7s (698 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 126210 files and directo

Upload GeoTIFF files from your local machine to Google Colab

In [ ]:
uploaded = files.upload()

In [ ]:
# Define paths
path_grib = "/content/"  # Change to your directory
path_gdal = "/content/"  # Change to your directory

#To verify uploaded files, use the following:
os.listdir()


['.config',
 'MRMS_GaugeCorr_QPE_01H_00.00_20190527-010000.tif',
 'MRMS_GaugeCorr_QPE_01H_00.00_20190527-010000.grib2',
 'sample_data']

In [ ]:
# Find all .grib2 files
files = glob.glob(f'{path_grib}*.grib2')
print(files)

['/content/MRMS_GaugeCorr_QPE_01H_00.00_20190527-010000.grib2']


MRMS handles two types of missing values (Missing and No Coverage). Depending on the MRMS variable you are using, these values may change. Check the documentation for the product you are using here: [MRMS GRIB2 Tables](https://www.nssl.noaa.gov/projects/mrms/operational/tables.php) and update those values accordingly.

In [ ]:
missing = -1
no_coverage = -3

In [ ]:
file = files[0] #Because files is an array of strings

output_name = os.path.basename(file)[:-6] + '.tif'  # Remove .grib2 extension
output_path = os.path.join(path_gdal, output_name)
temp_output_path = output_path.replace('.tif', '_temp.tif')  # Temporary file

try:
    # Step 1: Convert .grib2 to .tif
    command_translate = f"gdal_translate -of GTiff -a_nodata -9999 -ot Float32 -a_srs EPSG:4326 {file} {temp_output_path}"
    subprocess.run(command_translate, shell=True, check=True)

    # Step 2: Replace missing and no coverage data with -9999 using gdal_calc.py
    command_calc = (
        f"gdal_calc.py --overwrite --quiet "
        f"-A {temp_output_path} --outfile={output_path} "
        f"--calc='numpy.where((A=={missing}) | (A=={no_coverage}) | (A==-9999), -9999, A)' "
        f"--NoDataValue=-9999 --type=Float32"
    )
    subprocess.run(command_calc, shell=True, check=True)

    # Remove the temporary file
    os.remove(temp_output_path)

    print(f'File converted and corrected: {output_name}')

except Exception as e:
    print(f"Error processing file {file}: {str(e)}")


File converted and corrected: MRMS_GaugeCorr_QPE_01H_00.00_20190527-010000.tif


In [ ]:
  output_name = os.path.basename(file)[:-6] + '.tif'  # Remove .grib2 extension

  output_path = os.path.join(path_gdal, output_name)

  temp_output_path = output_path.replace('.tif', '_temp.tif')  # Temporary file



  try:

     # Step 1: Convert .grib2 to .tif

     command_translate = f"gdal_translate -of GTiff -a_nodata -9999 -ot Float32 -a_srs EPSG:4326 {file} {temp_output_path}"

     subprocess.run(command_translate, shell=True, check=True)



     print(f'File converted and corrected: {output_name}')



  except Exception as e:

     print(f"Error processing file {file}: {str(e)}")